In [71]:
import torch
import torch.nn as nn
import torchvision

In [ ]:
class AttrModel(nn.Module):
    def __init__(self, num_classes=294):
        super(AttrModel, self).__init__()
        self.num_classes = num_classes
        Base_Model = torchvision.models.resnet18(pretrained=True)
        
        children = list(Base_Model.children())
        
        self.conv = children[0]
        self.batch_norm = children[1]
        self.relu = children[2]
        self.maxpool = children[3]
        self.blocks1 = children[4]
        self.blocks2 = children[5]
        self.blocks3 = children[6]
        self.blocks4 = children[7]
        self.adp_avg_pool = children[8]
        self.fc = nn.Linear(children[9].in_features, self.num_classes)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.blocks1(x)
        x = self.blocks2(x)
        x = self.blocks3(x)
        x = self.blocks4(x)
        x = self.adp_avg_pool(x)
        x = self.fc(x)
        x = torch.nn.functional.sigmoid(x)
        return x

In [140]:
class AttrModel(nn.Module):
    def __init__(self, num_classes=294):
        super(AttrModel, self).__init__()
        self.num_classes = num_classes
        Base_Model = torchvision.models.resnet18(pretrained=True, progress=True)
        Base_Model.fc = nn.Linear(Base_Model.fc.in_features, num_classes)
        self.Model_Backend = Base_Model
        
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        print(x.shape)
        x = self.Model_Backend(x)
        print(x.shape)
        x = self.sig(x)
        return x

In [148]:
Att = AttrModel()
Att.train()

AttrModel(
  (Model_Backend): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [154]:
o = Att(torch.rand(10,3,100,100))
o

torch.Size([10, 3, 100, 100])
torch.Size([10, 294])


tensor([[0.3510, 0.3947, 0.7541,  ..., 0.6782, 0.4870, 0.7813],
        [0.6410, 0.4899, 0.8003,  ..., 0.5123, 0.4627, 0.6466],
        [0.6214, 0.5353, 0.7485,  ..., 0.4759, 0.4854, 0.5809],
        ...,
        [0.4537, 0.6379, 0.6516,  ..., 0.4429, 0.4423, 0.6444],
        [0.7183, 0.5425, 0.7870,  ..., 0.6191, 0.5069, 0.7072],
        [0.6232, 0.4676, 0.8873,  ..., 0.6576, 0.7108, 0.6965]],
       grad_fn=<SigmoidBackward>)

In [155]:
g = torch.rand((10,294))
g

tensor([[0.9108, 0.0874, 0.1864,  ..., 0.3081, 0.0518, 0.9199],
        [0.3871, 0.7088, 0.1673,  ..., 0.5470, 0.9612, 0.2184],
        [0.7549, 0.4777, 0.3052,  ..., 0.2691, 0.6178, 0.3612],
        ...,
        [0.2250, 0.6512, 0.8231,  ..., 0.9362, 0.4494, 0.4298],
        [0.1807, 0.7027, 0.9103,  ..., 0.4157, 0.9518, 0.9054],
        [0.9181, 0.5390, 0.6622,  ..., 0.6936, 0.4230, 0.5612]])

In [152]:
criterion = torch.nn.BCELoss()

In [158]:
l = criterion(o, g)
l

tensor(0.7543, grad_fn=<BinaryCrossEntropyBackward>)

In [157]:
params = [p for p in Att.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)

In [159]:
optimizer.zero_grad()
l.backward()
optimizer.step()